In [382]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re


In [383]:
# open data set

games = pd.read_csv('games.csv')


In [384]:
# remove unwanted columns

games_cols = []

for col in games.columns:
    games_cols.append(col)
    
print(games_cols)
    
wanted_cols = ['winner',
               'white_rating',
               'black_rating',
               'opening_eco'
              ]
# we only want the winners, ratings, and openings
games = games[wanted_cols]
games

['id', 'rated', 'created_at', 'last_move_at', 'turns', 'victory_status', 'winner', 'increment_code', 'white_id', 'white_rating', 'black_id', 'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply']


winner  white_rating  black_rating opening_eco
0      white          1500          1191         D10
1      black          1322          1261         B00
2      white          1496          1500         C20
3      white          1439          1454         D02
4      white          1523          1469         C41
...      ...           ...           ...         ...
20053  white          1691          1220         A80
20054  black          1233          1196         A41
20055  white          1219          1286         D00
20056  white          1360          1227         B07
20057  black          1235          1339         D00

[20058 rows x 4 columns]

In [385]:
## Scrape openings from https://www.365chess.com/eco.php so we can edit it onto our data set and get desired openings

# get ecos
eco_url = 'https://www.365chess.com/eco.php'
page = requests.get(eco_url)

# parse eco page
soup = BeautifulSoup(page.content, "html.parser")

# scrape ecos
x = soup.find_all(class_ = "opname")


# get raw ecos html
opnames = []
for y in x:
    opnames.append(y.text)
    

# parse raw ecos html into dict
opening_dict = {}

for i in opnames:
    split = i.split()
    if split[0] not in opening_dict:
        opening_dict[split[0]] = ' '.join(split[1:])

# only entry that's messed up, just manually fix it
opening_dict['D50-D69'] = "Queen's Gambit Declined"

In [386]:
## clean dict of ecos and subsets of major openings 

# get range of '-'
def get_range(a):
    num_range = []
    for x in a:
        num_range.append(int(re.sub('\D', '', x)))
    return num_range
        

# create new entries based on major openings and delete old ones
numbers = []
new_dict = opening_dict.copy()

for key in opening_dict:
    if '-' in key:
        numbers = (key.split('-'))
        # num_range = 69
        num_range = get_range(numbers)
        # num_ranges = 10 - 00
        num_ranges = num_range[1] - num_range[0] + 1
        alpha_key = numbers[0][0]
        for x in range(num_ranges):
            num_str = str((num_range[0] + x))
            if len(num_str) < 2:
                num_str = alpha_key + '0' + num_str
            else:
                num_str = alpha_key + num_str
            new_dict[num_str] = opening_dict[key]
        del new_dict[key]
    


op_eco_dict = collections.OrderedDict(sorted(new_dict.items()))

In [398]:
# successfully replace opening_eco
games = games.replace({'opening_eco' : op_eco_dict}) 
games = games.rename(columns = {'opening_eco': 'opening'})

games['rating diff'] = abs(games['white_rating'] - games['black_rating'])

# split by winner color now
black_win = games[games['winner'] == 'black'].reset_index(drop = True)
white_win = games[games['winner'] == 'white'].reset_index(drop = True)

